# Ingesta de cuestionarios


En este script, vamos a filtrar los cuestionarios de la base de datos de moodle para quedarnos solo con aquellos pertenecientes a la asginatura IP, y obtener solo la información que nos sea util de estas tablas.



## Configuración 


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import count
from itertools import count


import os

# Configuración
curso_ip = 8683
ruta_origen = "/home/carlos/Documentos/TFG/spark-workspace/data/raw"
ruta_destino = "/home/carlos/Documentos/TFG/spark-workspace/data/raw/ip"
os.makedirs(ruta_destino, exist_ok=True)

# Crear sesión Spark
spark = SparkSession.builder \
    .appName("Filtrado datos curso IP") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "8") \
    .getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/05/31 17:56:42 WARN Utils: Your hostname, carlos-Modern-15-A11SB, resolves to a loopback address: 127.0.1.1; using 192.168.1.182 instead (on interface wlo1)
25/05/31 17:56:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/31 17:56:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Carga y visualización de datos de partida

In [ ]:

# Cargar datos
quizes = spark.read.parquet(f"{ruta_origen}/quiz_cmi.parquet")
attempts = spark.read.parquet(f"{ruta_origen}/quiz_attempts_cmi.parquet")

print("=========quiz===============================")

quizes.printSchema()
quizes.show(5, truncate=False)
print("=========quiz_attempts===============================")
attempts.printSchema()
attempts.show(5, truncate=False)


=========quiz===============================
root
 |-- id: long (nullable = true)
 |-- course: long (nullable = true)
 |-- timeopen: long (nullable = true)
 |-- timeclose: long (nullable = true)
 |-- name: string (nullable = true)
 |-- timemodified: long (nullable = true)

+----+------+--------+----------+--------------------------+------------+
|id  |course|timeopen|timeclose |name                      |timemodified|
+----+------+--------+----------+--------------------------+------------+
|2601|2677  |0       |0         |CUESTIONARIO              |1286204734  |
|2603|2677  |0       |2147483647|mi cuestionario del tema 1|1170958009  |
|2605|2677  |0       |0         |Cuestionario              |1171213742  |
|2607|2677  |0       |0         |CUESTIONARIO              |1224176555  |
|2609|2677  |0       |0         |CUESTIONARIO              |1191603192  |
+----+------+--------+----------+--------------------------+------------+
only showing top 5 rows
=========quiz_attempts==============

## Filtrado de datos 


En primer lugar, vamos a quedarnos solo con aquellos cuestionarios de la asignatura IP

In [4]:
quizes_ip = quizes.filter(col("course") == curso_ip)

quizes_ip.show(1000, truncate=False)

+-----+------+----------+----------+-------------------------------------------------------------------+------------+
|id   |course|timeopen  |timeclose |name                                                               |timemodified|
+-----+------+----------+----------+-------------------------------------------------------------------+------------+
|14457|8683  |1695123000|0         |Actividad 01: Test sobre expresiones y programas simples           |1697623785  |
|16011|8683  |1699360200|1700261700|Actividad 06. Test sobre complejidad                               |1699985605  |
|16681|8683  |1694507400|0         |Cuestionario previo a la clase 1                                   |1697623785  |
|16685|8683  |1694593800|0         |Cuestionario previo a la clase 2                                   |1697623785  |
|16709|8683  |0         |0         |Cuestionario previo a la clase 05                                  |1697623785  |
|16720|8683  |0         |0         |Cuestionario previo 

Ahora, vamos a concatenar esta tabla con los intentos mediante un `INNER JOIN`, de tal modo que nos quedaremos solo con aquellos cuestionarios que tengan al menos una entrega asociada, y también solo con los intentos de cuestionarios que estén asociados a un quiz válido. 

De este modo, el resultado será un dataframe que tendrá información de todas las entregas hechas en el curso, en el que cada entrega tendrá concatenada información del cuestionario al que estaba asociado.

In [ ]:
from pyspark.sql.functions import count as spark_count


quiz_attempts_ip = quizes_ip.join(attempts, quizes_ip.id == attempts.quiz, "inner") \
      .select(col("id").alias("quiz_id"), "name", "timeopen", "timeclose", "userid", "state", "attempt", "sumgrades", "timestart", "timefinish") # Atributos que nos interesa almacenar
quiz_attempts_ip.printSchema()

quiz_attempts_ip.show(5, truncate=False)
 

root
 |-- quiz_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- timeopen: long (nullable = true)
 |-- timeclose: long (nullable = true)
 |-- userid: string (nullable = true)
 |-- state: string (nullable = true)
 |-- attempt: long (nullable = true)
 |-- sumgrades: string (nullable = true)
 |-- timestart: long (nullable = true)
 |-- timefinish: long (nullable = true)

+-------+--------------------------------+----------+---------+----------------------------------------------------------------+--------+-------+---------+----------+----------+
|quiz_id|name                            |timeopen  |timeclose|userid                                                          |state   |attempt|sumgrades|timestart |timefinish|
+-------+--------------------------------+----------+---------+----------------------------------------------------------------+--------+-------+---------+----------+----------+
|16681  |Cuestionario previo a la clase 1|1694507400|0        |5220ebcc6c2637b

Veamos cuantos cuestionarios han pasado la condición de join, y cuantas entregas tiene asociado cada cuestionario

In [ ]:

attemps_per_quiz = quiz_attempts_ip.groupBy("quiz_id", "name").agg(count("*").alias("num_attempts")).orderBy("quiz_id")

attemps_per_quiz.show(1000, truncate=False)

print(f"Número de cuestionarios inicial: {quizes_ip.count()} ")
print(f"Número de cuestionarios tras quedarnos con aquellos que tienen asociada al menos una entrega: {attemps_per_quiz.count()}")

+-------+-------------------------------------------------------------------+------------+
|quiz_id|name                                                               |num_attempts|
+-------+-------------------------------------------------------------------+------------+
|14457  |Actividad 01: Test sobre expresiones y programas simples           |252         |
|16011  |Actividad 06. Test sobre complejidad                               |278         |
|16681  |Cuestionario previo a la clase 1                                   |208         |
|16685  |Cuestionario previo a la clase 2                                   |200         |
|16709  |Cuestionario previo a la clase 05                                  |211         |
|16720  |Cuestionario previo a la clase 6                                   |229         |
|16750  |Cuestionario previo a la clase 10                                  |186         |
|16804  |Cuestionario previo a la clase 13                                  |187         |

Podemos ver como han desaparecido 8 cuestionarios que estaban asociados al aula, pero que realmente no tenían ninguna entrega, así que han dejado de aparecer tras concatenar ambas tablas. 

Ahora, el siguiente paso será quedarnos solo con los cuestionarios y entregas las cuales hayan sido realizadas por estudiantes matrículados en la asignatura.

In [ ]:


students_ip = spark.read.parquet(f"{ruta_origen}/alumnos_ip_cmi.parquet")

quiz_attempts_ip_st = quiz_attempts_ip.join(students_ip, ["userid"], "inner")

attemps_per_quiz_st = quiz_attempts_ip_st.groupBy("quiz_id", "name").agg(count("*").alias("num_attempts")).orderBy("quiz_id")

attemps_per_quiz_st.show(1000, truncate=False)

print(f"Número de cuestionarios tras quedarnos con aquellos que tienen asociada al menos una entrega y un alumno: {attemps_per_quiz_st.count()}")




+-------+-------------------------------------------------------------------+------------+
|quiz_id|name                                                               |num_attempts|
+-------+-------------------------------------------------------------------+------------+
|14457  |Actividad 01: Test sobre expresiones y programas simples           |248         |
|16011  |Actividad 06. Test sobre complejidad                               |274         |
|16681  |Cuestionario previo a la clase 1                                   |203         |
|16685  |Cuestionario previo a la clase 2                                   |197         |
|16709  |Cuestionario previo a la clase 05                                  |208         |
|16720  |Cuestionario previo a la clase 6                                   |223         |
|16750  |Cuestionario previo a la clase 10                                  |177         |
|16804  |Cuestionario previo a la clase 13                                  |180         |

Podemos ver que no ha desaparecido ningún cuestionario tras aplicar esta unión, simplemente si que se ha reducido un poco el número de entregas que tiene cada cuestionario, quizás por que algunas de las entregas de los cuestionarios fueran hechos por estudiantes que no estaban ofucialmente matriculados o a lo mejor por haber sido de usuarios con rol de profesor que simplemente probaron el cuestionario tras crearlo para verificar que estuviera correcto. 


## Exploración final de los datos

Vamos a hacer alguna comprobación extra a los datos antes de almacenarlos y comenzar a calcular métricas sobre ellos, como cuantas de las entregas tienen la columna state con el valor finished y con los otros valores posibles, y ver si hay mucha diferencia entre la cantidad de entregas que ha hecho cada usuario registrado en la tabla

In [ ]:
print("Cuantos entregas hay asociadas a cada tipo de intento de cuestionario en base a su estado:\n")

kinds_of_attempt = quiz_attempts_ip_st.groupBy("state").agg(spark_count("*").alias("count"))

kinds_of_attempt.show(1000, truncate=False)

print("Cantidad de entregas hechas por alumno en el curso:\n")
attempts_per_student = quiz_attempts_ip_st.groupBy("userid").agg(spark_count("*").alias("num_attempts")).orderBy("userid")
attempts_per_student.show(1000, truncate=False)

print("Alumnos que superan la cantidad de 25 entregas de cuestionarios\n")
attempts_per_student.filter(col("num_attempts") > 25).show(1000, truncate=False)

print("Media de notas de los alumnos\n")


Cuantos entregas hay asociadas a cada tipo de intento de cuestionario en base a su estado:

+----------+-----+
|state     |count|
+----------+-----+
|inprogress|16   |
|finished  |2391 |
|abandoned |7    |
+----------+-----+

Cantidad de entregas hechas por alumno en el curso:

+----------------------------------------------------------------+------------+
|userid                                                          |num_attempts|
+----------------------------------------------------------------+------------+
|006b0e7bd07cec05e0952cb61c30893f6d30d7962f9efc99d0f041f6fadcc320|8           |
|00ded60939d4949cc46e46e865b25d3f11756733cf946087710c61eda02729e1|13          |
|05912200993a87a89df1a6ca9ac3d6493e2c4cc178760d8ee1da41033ac01b3e|13          |
|073b1d0ee1d3857d50ea87087b25bbc6f5dbdbd2e94bcf52b89c48afa37e8c16|22          |
|080b2c8b65e9d941f12e62b7d2b9fa22b669f06aeed07df5683fdf93a799204d|3           |
|091af124e119a447c7f6594fb2f7c4fbb678f669966db01e3f62c26eedb220af|12          |
|

Podemos ver que , la distribución de valores entre entregas de cada tipo en base a su tipo es la esperada, dado que las que mas hay son las de tipo finished. Por otra parte, podemos ver que el número de intentos llevado a cabo por cada usuario también lo es, dado que la distribución de los valores obtenidos para cada usuario están alineados con la cantidad de cuestionarios que hubo publicados  a lo largo del curso en el aula. 

## Exportar a parquet el conjunto de datos obtenido

In [27]:
quiz_attempts_ip_st.write.mode("overwrite").parquet(f"{ruta_destino}/quiz_attempts_st_ip.parquet")